In [1]:
import pandas as pd
import numpy as np
import sklearn

import warnings
warnings.filterwarnings("ignore")

from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score

N_FOLDS = 3

In [11]:
train_data_def = pd.read_csv('train/train.csv', sep=';')

train_start = '2020-06'
test_start = '2020-07'

train_data = train_data_def[train_data_def['order_completed_at']==train_start]
print(train_data['phone_id'].nunique())
print(train_data['target'].value_counts())

test_data = train_data_def[train_data_def['order_completed_at']==test_start]
print(test_data['phone_id'].nunique())
print(test_data['target'].value_counts())

train_data.loc[:, 'fold_num'] = [np.random.randint(1,N_FOLDS+1) for i in range(len(train_data))]
train_data['fold_num'].value_counts()
train_data['month']=6

test_data.loc[:, 'fold_num'] = [np.random.randint(1,N_FOLDS+1) for i in range(len(test_data))]
test_data['fold_num'].value_counts()
test_data['month']=7


train_folds = []
test_folds = []

for i in range(1, N_FOLDS+1):
    local_data_train = train_data.loc[train_data['fold_num']==i]
    local_data_test = test_data.loc[test_data['fold_num']==i]

    del local_data_train['fold_num']
    del local_data_test['fold_num']
    
    del local_data_train['order_completed_at']
    del local_data_test['order_completed_at']
    
    train_folds.append(local_data_train.reset_index(drop=True))
    test_folds.append(local_data_test.reset_index(drop=True))


229989
0.0    115137
1.0    114852
Name: target, dtype: int64
257197
0.0    136768
1.0    120429
Name: target, dtype: int64


In [12]:
def get_X_y(folds, i, fea):
    local_fea = train_folds[i].merge(fea, on =['phone_id', 'month'])
    del local_fea['phone_id']
    del local_fea['month']
    return local_fea.drop('target', axis=1).values, local_fea['target'].values

In [18]:
##JUST AN EXAMPLE OF FEA
# fea = pd.DataFrame([[19843],[7], [1], [3]]).T
# fea.columns = ['phone_id', 'month', 'f1', 'f2']
#END OF EXAMPLE FEA
fea = pd.read_csv('fea/f_shipments.csv')

clf = SGDClassifier()

scores = []
for i in range(N_FOLDS):
    
    print(i)
    X_tr, y_tr = get_X_y(train_folds, i, fea)
    X_te, y_te = get_X_y(test_folds, i, fea)    
    
    clf.fit(X_tr, y_tr)
    preds = clf.predict(X_te)
    
    scores.append(f1_score(y_te, preds))
    break
    
    
print(f'mean score - {np.mean(scores)}')

0
mean score - 0.7817942605546655
